In [ ]:
# dependecies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.base import BaseEstimator, RegressorMixin
import tensorflow as tf
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor

PREPROCESSING/EDA

In [ ]:
# reading in the data
data = pd.read_csv('/tmp/insurance.csv')
data.head()

In [ ]:
# understanding categorical vs numerical (can also use df.dtypes)
data.info()

In [ ]:
# checking null values
data.isnull().sum()

In [ ]:
# checking duplicated values
data.duplicated().sum()

In [ ]:
duplicated_rows = data[data.duplicated()]
duplicated_rows

In [ ]:
# handling duplicates
data = data.drop_duplicates()

In [ ]:
data.shape

don't do label encoding which is assigning a number to each feature; it messes up a neural network; changing to one hot encoding which makes dummy variables for each feature (sql distinct comes in clutch here) and then uses binary

In [ ]:
ohe_region = pd.get_dummies(data.region)
ohe_sex = pd.get_dummies(data.sex)
ohe_smoker = pd.get_dummies(data.smoker)

ADVANCED FEATURE ENGINEERING

In [ ]:
# numerical => categorical for "imbalance"/skew purposes; THIS IS CALLED BINNING
data['age_category'] = pd.cut(data['age'], bins=[0, 25, 35, 45, 55, 65], labels=['18-25', '26-35', '36-45', '46-55', '56-65'])
data['bmi_category'] = pd.cut(data['bmi'], bins=[0, 18.5, 25, 30, np.inf], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
data['charges_category'] = pd.cut(data['charges'], bins=[0, 10000, 20000, 30000, 40000, 50000, 60000, np.inf], labels=['$0-10,000', '$10,001-20,000', '$20,001-30,000', '$30,001-40,000', '$40,001-50,000', '$50,001-60,000', '$60,000+'])

In [ ]:
# focusing on smokers as correlation matrix indicated strong relationship
data['elderly_smoker'] = ((data['smoker'] == 'yes') & (data['age'] > 39)).astype(int)
data['obese_smoker'] = ((data['smoker'] == 'yes') & (data['bmi_category'] == 'Obese')).astype(int)

# family size impact (testing)
data['has_children'] = ((data['children'] > 0)).astype(int)
data['largeFamily'] = ((data['children'] >= 3)).astype(int)

# MORE
data['age_squared'] = data['age'] ** 2
data['log_bmi'] = np.log1p(data['bmi'])
data['log_charges'] = np.log1p(data['charges'])

In [ ]:
data = pd.concat([data, ohe_region, ohe_sex, ohe_smoker], axis='columns')
data.head()

In [ ]:
data.to_csv('cleaned_data.csv', index=False)

BEGINNING THE VISUALIZATIONS

In [ ]:
# for all my plots
palette = [ '#C66F80', '#F4C7D0','#FCEBF1', '#4A6644', '#9FAA74', '#D7DAB3', '#ECE3D2']
customCmap = ListedColormap(palette)

In [ ]:
# correlation matrix
correlation_matrix = data.select_dtypes(include=["number", "bool"]).corr()
plt.figure(figsize=(12,10))
sns.heatmap(correlation_matrix, annot=True, cmap=customCmap, 
            fmt='.3f', square=True, linewidths=0.5)
plt.title('Correlation Matrix - All Numeric & Boolean Features',)
plt.tight_layout()
plt.show()

print("\nCorrelations with Charges (target):")
print(correlation_matrix['charges'].sort_values(ascending=False))

strong correlation between smoking & insurance charges

In [ ]:
# histograms
histogram = data.select_dtypes(include=["number", "bool"]).hist(bins=15, figsize=[15,10])
plt.show()

In [ ]:
data[data['yes'] == True]['charges'].hist(bins=15, figsize=[8,6])


age, children, and charges are skewed to the right. bmi is reminscent of a normal curve. There is evidence of imbalance in age, children, and smoker features.

In [ ]:
# due to charge feature skew, most outliers are on the higher end. 
sns.boxplot(y='charges', x='region', hue='smoker', data=data, palette=palette)

multiple experiments with boxplots; can make numerous conclusions about them. 

In [ ]:
# pairplot video experimentation; this is so large bc of the boolean & numerical features; not super important in the grand scheme of things
sns.pairplot(data, kind='hist')

- detecting numerical features
- histograms on the diagonals; scatter plots everywhere else; can be changed
- use hue for categorical features
- boolean types are treated as numeric
- can specify which exact variables you want to see & which axis they're on 

In [ ]:
sns.pairplot(data, hue='smoker', palette=palette, vars=['charges', 'bmi'])

In [ ]:
# to understand a misconception
sns.kdeplot(data, x='age', hue='smoker', palette=palette)

learned quite a lot from that; despite the normal-looking curve for bmi, the results showed heavy skew for overweight & obese individuals

In [ ]:
sns.boxplot(y='charges', x='sex', hue='bmi_category', data=data, palette=palette)

In [ ]:
data.head()

In [ ]:
# let's create a contingency table to see sex & bmi broken down
crosstab01 = pd.crosstab(data['sex'], data['bmi_category'])
crosstab01

In [ ]:
# visualization of above
crosstab01.plot(kind='bar', stacked = True, colormap=customCmap)

In [ ]:
# checking feature importance (i think??); seeing if i can see any distinct noticeable patterns
columns = ['age_category', 'sex', 'bmi_category', 'smoker', 'children', 'region']

for col in columns:
    plt.figure(figsize=(8, 4))
    sns.countplot(x=col, hue='charges_category', data=data, palette=palette)
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.title('Insurance Charges', loc='right')
    plt.show()

In [ ]:
# interesting but knew from sql queries as well as really only helpful for visualizations & classification problems
data['charges_category'].value_counts().plot(kind='bar', colormap=customCmap)

In [ ]:
# SO SKEWED
data['charges_category'].value_counts()

In [ ]:
# finding average features for each charge category
numeric_columns = ['age', 'children', 'bmi']

for col in numeric_columns:
    mean_values = data.groupby('charges_category')[col].mean()
    plt.figure(figsize=(12, 6))
    mean_values.plot(kind='bar', color=customCmap.colors)
    plt.title(f'Average {col.capitalize()} per Insurance Charge Category')
    plt.xlabel('Charge Category')
    plt.xticks(rotation=0)
    plt.ylabel(col.capitalize())
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

MODELING

In [ ]:
modeling_data = data.drop(['sex', 'smoker', 'region', 'charges_category', 'age_category', 'bmi_category', 'children', 'age', 'bmi', 'charges'], axis=1)
modeling_data.head()

In [ ]:
# feature & target split
x = modeling_data.drop('log_charges', axis=1)
y = modeling_data['log_charges']

# test train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
y_train = scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test = scaler.transform(y_test.values.reshape(-1, 1)).flatten()

going to compare random forest (regressor) & regression neural network & decision tree (regressor)

In [ ]:
# starting with decision tree
dt_model = DecisionTreeRegressor(max_depth=5, min_samples_split=2, min_samples_leaf=10, criterion='absolute_error', random_state=42)

In [ ]:
dt_model.fit(x_train, y_train)

In [ ]:
y_pred_dt = dt_model.predict(x_test)
y_pred_dt = scaler.inverse_transform(y_pred_dt.reshape(-1, 1))
y_pred_dt = np.expm1(y_pred_dt)

y_true_dt = scaler.inverse_transform(y_test.reshape(-1, 1))
y_true_dt = np.expm1(y_true_dt)

In [ ]:
mse_dt = mean_squared_error(y_true_dt, y_pred_dt)
mae_dt = mean_absolute_error(y_true_dt, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)
r2_dt = r2_score(y_true_dt, y_pred_dt)

print(f"Decision Tree - Mean Squared Error: {mse_dt:.4f}")
print(f"Decision Tree - Mean Absolute Error: {mae_dt:.4f}")
print(f"Decision Tree - Root Mean Squared Error: {rmse_dt:.4f}")
print(f"Decision Tree - R² Score: {r2_score(y_true_dt, y_pred_dt):.4f}")

In [ ]:
data.describe()

In [ ]:
mean_charges = 13279
mae_pct_dt = mae_dt / mean_charges * 100
rmse_pct_dt = rmse_dt / mean_charges * 100
print(f"MAE is about {mae_pct_dt:.1f}% of the average cost")
print(f"RMSE is about {rmse_pct_dt:.1f}% of the average cost")

In [ ]:
# feature importance for decision forests
importance = dt_model.feature_importances_
x_names = ['elderly_smoker', 'obese_smoker', 'has_children', 'largeFamily',	'age_squared',	'log_bmi',	'northeast',	'northwest',	'southeast',	'southwest',	'female',	'male',	'no',	'yes']
feature_importances = list(zip(x_names, importance))
feature_importances_sorted = sorted(feature_importances, key=lambda x: x[1], reverse=True)
sorted_names, sorted_importance = zip(*feature_importances_sorted)

plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_importance)), sorted_importance, align='center', color=palette)
plt.yticks(np.arange(len(sorted_importance)), sorted_names)
plt.gca().invert_yaxis()
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance of Decision Tree Algorithm')
plt.show()

In [ ]:
# adding residual plot
y_pred_dt = np.ravel(y_pred_dt)
y_true_dt = np.ravel(y_true_dt)
residuals = y_true_dt - y_pred_dt

plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_pred_dt, y=residuals)
plt.axhline(0, color='pink', linestyle='--')
plt.xlabel('Predicted Charges')
plt.ylabel('Residuals')
plt.title('Residual Plot of Decision Tree Algorithm')
plt.show()

In [ ]:
# adding actual vs predicted plot
plt.scatter(y_true_dt, y_pred_dt, alpha = 0.4)
plt.plot([y_true_dt.min(), y_true_dt.max()], [y_true_dt.min(), y_true_dt.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()

In [ ]:
# beginning of error analysis
errors = y_true_dt.flatten() - y_pred_dt.flatten()
abs_errors = np.abs(errors)
pct_errors = (abs_errors / y_true_dt.flatten()) * 100

# create dataframe
error_data_dt = pd.DataFrame({
    'true_value': y_true_dt.flatten(),
    'predicted_value': y_pred_dt.flatten(),
    'error': errors,
    'abs_error': abs_errors,
    'pct_error': pct_errors
})

x_test_original = pd.DataFrame(
    scaler.inverse_transform(x_test),
    columns = x_names
)

error_data_dt = pd.concat([error_data_dt, x_test_original.reset_index(drop = True)], axis = 1)

In [ ]:
error_data_dt.to_csv('errors_dt.csv')

In [ ]:
error_data_dt.head()

In [ ]:
# most basic error statistics
error_data_dt.describe()

In [ ]:
# missing error statistics
print(f"median error: ${np.median(errors):,.2f}")
print(f"median absolute error: ${np.median(abs_errors):,.2f}")

In [ ]:
# error percentiles
print('=== error percentiles ===')
for percentile in [50, 75, 90, 95, 99]:
    value = np.percentile(abs_errors, percentile)
    print(f'{percentile}th percentile: ${value:,.2f}')

In [ ]:
# check for bias (systematic open/under prediction)
t_stat, p_value = stats.ttest_1samp(errors, 0)

print(f"mean error: ${errors.mean():.2f}")
print(f'as % of average cost: {(errors.mean() / y_true_dt.mean()) * 100:.2f}')

if p_value < 0.05 and abs(errors.mean()) > 100:
    if errors.mean() > 0:
        print(f"\n⚠️ model systematically underpredicts (p={p_value:.4f})")
    else: 
        print(f"\n⚠️ model systematically overpredicts (p={p_value:.4f})")
else:
    print(f"\n✅ No significant systematic bias detected (p={p_value:.4f})")

In [ ]:
# Are high-cost cases driving the bias?
print("\n=== Errors by Cost Range ===")
error_data_dt['cost_bin'] = pd.cut(error_data_dt['true_value'], 
                               bins=[0, 5000, 10000, 20000, 100000],
                               labels=['<$5k', '$5-10k', '$10-20k', '>$20k'])
print(error_data_dt.groupby('cost_bin')['error'].agg(['mean', 'count']))

# My guess: You're underpredicting HIGH-cost cases significantly
# Which pulls up the overall mean error

In [ ]:
# Check this
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(y_true_dt, errors, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('True Value ($)')
plt.ylabel('Error (True - Predicted) ($)')
plt.title('Error Pattern: Underprediction Increases with Cost')
plt.show()

# I bet you'll see errors trending upward as true value increases!

In [ ]:
# Let's look at this more carefully
print("=== Detailed Error Analysis by Cost Range ===\n")

for range_name in ['<$5k', '$5-10k', '$10-20k', '>$20k']:
    range_data = error_data_dt[error_data_dt['cost_bin'] == range_name]
    
    print(f"\n{range_name}:")
    print(f"  Count: {len(range_data)}")
    print(f"  Mean error: ${range_data['error'].mean():,.2f}")
    print(f"  Median error: ${range_data['error'].median():,.2f}")  # KEY METRIC!
    print(f"  Std dev: ${range_data['error'].std():,.2f}")
    print(f"  Max error: ${range_data['error'].max():,.2f}")
    print(f"  Min error: ${range_data['error'].min():,.2f}")
    
    # How many are actually way off?
    big_errors = range_data[np.abs(range_data['error']) > 5000]
    print(f"  # with error >$5k: {len(big_errors)} ({len(big_errors)/len(range_data)*100:.1f}%)")

In [ ]:
# Check this
print("\n=== Outlier Analysis ===")
threshold = 5000

outliers = error_data_dt[np.abs(error_data_dt['error']) > threshold]
print(f"Predictions with error >${threshold}: {len(outliers)} ({len(outliers)/len(error_data_dt)*100:.1f}%)")
print(f"These {len(outliers)} outliers account for ${outliers['abs_error'].sum():,.0f} of total error")
print(f"That's {outliers['abs_error'].sum() / error_data_dt['abs_error'].sum() * 100:.1f}% of total error")

print("\nWhat do outliers look like?")
print(outliers[['true_value', 'predicted_value', 'error']].describe())

In [ ]:
print("\n=== WHO ARE THE OUTLIERS? ===\n")

# Get the 24 outliers
outliers = error_data_dt[np.abs(error_data_dt['error']) > 5000]

print("Outlier Characteristics:")
if 'smoker_yes' in outliers.columns:
    print(f"\nSmoker percentage:")
    print(f"  Outliers: {outliers['smoker_yes'].mean()*100:.1f}%")
    print(f"  Overall: {error_df['smoker_yes'].mean()*100:.1f}%")

if 'bmi' in outliers.columns:
    print(f"\nAverage BMI:")
    print(f"  Outliers: {outliers['bmi'].mean():.1f}")
    print(f"  Overall: {error_df['bmi'].mean():.1f}")

if 'age' in outliers.columns:
    print(f"\nAverage Age:")
    print(f"  Outliers: {outliers['age'].mean():.1f}")
    print(f"  Overall: {error_df['age'].mean():.1f}")

print("\n=== HYPOTHESIS CHECK ===")
# My guess: outliers are mostly smokers with high BMI
if 'smoker_yes' in outliers.columns and 'bmi' in outliers.columns:
    high_risk = outliers[(outliers['smoker_yes'] == 1) & (outliers['bmi'] > 30)]
    print(f"Outliers that are obese smokers: {len(high_risk)} / {len(outliers)} ({len(high_risk)/len(outliers)*100:.1f}%)")

RANDOM FOREST

In [ ]:
# adding hyperparam tuning to random forest 
n_estimators = [50, 100, 200, 300] # num of trees in random forest
max_features = ['auto', 'sqrt', 'log2'] # num of features to consider @ every split
max_depth = [4, 8, 12, None] # max num of levels in tree
min_samples_split = [2, 5, 10] # min num of samples to split a node
min_samples_leaf = [1, 2, 4] # min num of samples to split each leaf node
bootstrap = [True] # method of selecting samples for tree training

In [ ]:
param_grid = {'n_estimators': n_estimators,
                            'max_features': max_features,
                            'max_depth': max_depth,
                            'min_samples_split': min_samples_split,
                            'min_samples_leaf': min_samples_leaf,
                            'bootstrap': bootstrap}

In [ ]:
rf_model = RandomForestRegressor(random_state = 42)
rf_random = RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 3, verbose = 2, n_jobs = 1, n_iter = 50, scoring = 'neg_mean_absolute_error', random_state = 42)
rf_random.fit(x_train, y_train)

In [ ]:
best_rf_model = rf_random.best_estimator_

In [ ]:
y_pred_rf = best_rf_model.predict(x_test)
y_pred_rf = scaler.inverse_transform(y_pred_rf.reshape(-1, 1))
y_pred_rf = np.expm1(y_pred_rf)

y_true_rf = scaler.inverse_transform(y_test.reshape(-1, 1))
y_true_rf = np.expm1(y_true_rf)

In [ ]:
mse_rf = mean_squared_error(y_true_rf, y_pred_rf)
mae_rf = mean_absolute_error(y_true_rf, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_true_rf, y_pred_rf)

print(f"Random Forest - Mean Squared Error: {mse_rf:.4f}")
print(f"Random Forest - Mean Absolute Error: {mae_rf:.4f}")
print(f"Random Forest - Root Mean Squared Error: {rmse_rf:.4f}")
print(f"Random Forest - R² Score: {r2_score(y_true_rf, y_pred_rf):.4f}")

In [ ]:
mae_pct_rf = mae_rf / mean_charges * 100
rmse_pct_rf = rmse_rf / mean_charges * 100
print(f"MAE is about {mae_pct_rf:.1f}% of the average cost")
print(f"RMSE is about {rmse_pct_rf:.1f}% of the average cost")

In [ ]:
# feature importance for random forests
importance = best_rf_model.feature_importances_
x_names = ['elderly_smoker', 'obese_smoker', 'has_children', 'largeFamily',	'age_squared',	'log_bmi',	'northeast',	'northwest',	'southeast',	'southwest',	'female',	'male',	'no',	'yes']
feature_importances = list(zip(x_names, importance))
feature_importances_sorted = sorted(feature_importances, key=lambda x: x[1], reverse=True)
sorted_names, sorted_importance = zip(*feature_importances_sorted)

plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_importance)), sorted_importance, align='center', color=palette)
plt.yticks(np.arange(len(sorted_importance)), sorted_names)
plt.gca().invert_yaxis()
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance of Random Forest Algorithm')
plt.show()

In [ ]:
# adding residual plots
y_pred_rf = np.ravel(y_pred_rf)
y_true_rf = np.ravel(y_true_rf)
residuals = y_true_rf - y_pred_rf

plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_pred_rf, y=residuals)
plt.axhline(0, color='pink', linestyle='--') 
plt.xlabel('Predicted Charges')
plt.ylabel('Residuals')
plt.title('Residual Plot of Random Forest Algorithm')
plt.show()

In [ ]:
# adding actual vs predicted plot
plt.scatter(y_true_rf, y_pred_rf, alpha = 0.4)
plt.plot([y_true_rf.min(), y_true_rf.max()], [y_true_rf.min(), y_true_rf.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()

In [ ]:
# beginning of error analysis
errors = y_true_rf.flatten() - y_pred_rf.flatten()
abs_errors = np.abs(errors)
pct_errors = (abs_errors / y_true_rf.flatten()) * 100

# create dataframe
error_data_rf = pd.DataFrame({
    'true_value': y_true_rf.flatten(),
    'predicted_value': y_pred_rf.flatten(),
    'error': errors,
    'abs_error': abs_errors,
    'pct_error': pct_errors
})

x_test_original = pd.DataFrame(
    scaler.inverse_transform(x_test),
    columns = x_names
)

error_data_rf = pd.concat([error_data_rf, x_test_original.reset_index(drop = True)], axis = 1)

In [ ]:
error_data_rf.to_csv('errors_rf.csv')

NEURAL NETWORK

In [ ]:
# simple nn arch
def build_nn(n_features):
    model = keras.Sequential([
        Input(shape=(n_features,)),
        tf.keras.layers.Dense(64, activation = 'relu', ),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])
    return model

In [ ]:
nn_model = KerasRegressor(model=build_nn, epochs=20, batch_size=16, verbose=0, model__n_features = x_train.shape[1])

In [ ]:
nn_model.fit(x_train, y_train)

In [ ]:
y_pred_nn = nn_model.predict(x_test)
y_pred_nn = scaler.inverse_transform(y_pred_nn.reshape(-1, 1))
y_pred_nn = np.expm1(y_pred_nn)

y_true_nn = scaler.inverse_transform(y_test.reshape(-1, 1))
y_true_nn = np.expm1(y_true_nn)

In [ ]:
mse_nn = mean_squared_error(y_true_nn, y_pred_nn)
mae_nn = mean_absolute_error(y_true_nn, y_pred_nn)
rmse_nn = np.sqrt(mse_nn)
r2_nn = r2_score(y_true_nn, y_pred_nn)

print(f"Neural Network - Mean Squared Error: {mse_nn:.4f}")
print(f"Neural Network - Mean Absolute Error: {mae_nn:.4f}")
print(f"Neural Network - Root Mean Squared Error: {rmse_nn:.4f}")
print(f"Neural Network - R² Score: {r2_score(y_true_nn, y_pred_nn):.4f}") # for PFI, this is baseline score; where no score has been permuted

In [ ]:
mae_pct_nn = mae_nn / mean_charges * 100
rmse_pct_nn = rmse_nn / mean_charges * 100
print(f"MAE is about {mae_pct_nn:.1f}% of the average cost")
print(f"RMSE is about {rmse_pct_nn:.1f}% of the average cost")

In [ ]:
# permutation feature importance for neural network
r = permutation_importance(nn_model, x_test, y_test, n_repeats = 10, random_state = 42)

sorted_idx = r.importances_mean.argsort()
plt.barh(np.array(x_names)[sorted_idx], r.importances_mean[sorted_idx], color = palette)
plt.xlabel("Permutation Importance")
plt.title("Feature Importance (Neural Network)")
plt.show()

In [ ]:
# adding residual plots
y_pred_nn = np.ravel(y_pred_nn)
y_true_nn = np.ravel(y_true_nn)
residuals = y_true_nn - y_pred_nn

plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_pred_nn, y=residuals)
plt.axhline(0, color='pink', linestyle='--') 
plt.xlabel('Predicted Charges')
plt.ylabel('Residuals')
plt.title('Residual Plot of Neural Network Algorithm')
plt.show()

In [ ]:
# adding actual vs predicted plot
plt.scatter(y_true_nn, y_pred_nn, alpha = 0.4)
plt.plot([y_true_nn.min(), y_true_nn.max()], [y_true_nn.min(), y_true_nn.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()

In [ ]:
# adding kfold cross val
kfold = KFold(n_splits = 5, shuffle=True, random_state = 42)
cv_scores = cross_val_score(nn_model, x_train, y_train, cv = kfold, scoring = 'r2')

print(f"Cross-Validation R² Scores: {cv_scores}")
print(f"Mean R²: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

In [ ]:
# beginning of error analysis
errors = y_true_nn.flatten() - y_pred_nn.flatten()
abs_errors = np.abs(errors)
pct_errors = (abs_errors / y_true_nn.flatten()) * 100

# create dataframe
error_data_nn = pd.DataFrame({
    'true_value': y_true_nn.flatten(),
    'predicted_value': y_pred_nn.flatten(),
    'error': errors,
    'abs_error': abs_errors,
    'pct_error': pct_errors
})

x_test_original = pd.DataFrame(
    scaler.inverse_transform(x_test),
    columns = x_names
)

error_data_nn = pd.concat([error_data_nn, x_test_original.reset_index(drop = True)], axis = 1)

In [ ]:
error_data_nn.to_csv('errors_nn.csv')